In [104]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import root_scalar

In [105]:
#Constants

R = 8.314
P = 1

A_but = 9.9614
B_but = 2664.0939
C_but = -104.881
A_h2O = 11.9647
B_h2O = 3984.9273
C_h2O = -39.734

g12_g22 = 1034.3
g21_g11 = 10098.50
aij = 0.4118

In [106]:
def molfrac_vapor(T, x):
    x_but = x
    x_h2O = 1 - x

    tau_12 = g12_g22 / (R * T)
    tau_21 = g21_g11 / (R * T)

    G_12 = np.exp(-aij * tau_12)
    G_21 = np.exp(-aij * tau_21)

    gamma_but = np.exp(x_h2O**2 * (tau_21 * (G_21 / (x_but + x_h2O * G_21))**2 + (tau_12 * G_12) / (x_h2O + x_but * G_12)**2))
    gamma_h2O = np.exp(x_but**2 * (tau_12 * (G_12 / (x_h2O + x_but * G_12))**2 + (tau_21 * G_21) / (x_but + x_h2O * G_21)**2))
    
    Psat_but = np.exp(A_but - B_but / (T + C_but))
    Psat_h2O = np.exp(A_h2O - B_h2O / (T + C_h2O))
    
    y_but = gamma_but * x_but * Psat_but / P
    y_h2O = gamma_h2O * x_h2O * Psat_h2O / P
    
    sum_y = y_but + y_h2O
    
    return sum_y - 1, y_but, y_h2O

In [107]:
x_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
T_solutions = []
y_buts = []
y_h2Os = []
initial_guess = 373

for i,x in enumerate(x_values):
    sol, y_but, y_h20 = fsolve(molfrac_vapor, initial_guess, x)
    T_solutions.append(sol)
    y_buts.append(y_but)
    y_h2Os.append(y_h20)

print("Solutions for T:", T_solutions)
print("Solutions for y_but:", y_buts)
print("Solutions for y_h2O:", y_h2Os)
print(f"mean: {np.mean(T_solutions)}")

ValueError: object too deep for desired array

error: Result from function call is not a proper array of floats.